In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /home/edu/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpxsu7z1zw
  JVM stdout: /tmp/tmpxsu7z1zw/h2o_edu_started_from_python.out
  JVM stderr: /tmp/tmpxsu7z1zw/h2o_edu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_edu_xlu26m
H2O cluster total nodes:,1
H2O cluster free memory:,1.887 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [2]:
train_high = h2o.import_file('../cleaned-data/05-train_high-catAsDummies-withXYZ.csv')
train_low = h2o.import_file('../cleaned-data/05-train_low-catAsDummies-withXYZ.csv')
test_high = h2o.import_file('../cleaned-data/05-test_high-catAsDummies-withXYZ.csv')
test_low = h2o.import_file('../cleaned-data/05-test_low-catAsDummies-withXYZ.csv')      
train_high.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


id,carat,depth,table,x,y,z,cut_0,cut_1,cut_2,cut_3,cut_4,color_1,color_2,color_3,color_4,color_5,color_6,color_7,clarity_0,clarity_1,clarity_2,clarity_3,clarity_4,clarity_5,clarity_6,clarity_7,price
8,2.05,62.4,57,8.17,8.09,5.08,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,18193
34,2.01,64.9,60,7.82,7.78,5.06,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,13744
42,2,64.2,52,7.91,7.88,5.07,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,11400
72,2,60.3,56,8.08,8.11,4.88,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,16425
82,2.28,62.7,57,8.43,8.36,5.26,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,16450
93,2.02,60.4,58,8.2,8.16,4.94,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,17357
141,2,60.7,60,8.07,8.11,4.91,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,15312
154,2.01,61.5,62,8.07,7.94,4.92,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,15475
165,2.31,62.9,58,8.43,8.34,5.27,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,17218
167,2.01,60.6,57,8.19,8.09,4.92,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,17383


In [3]:
list(map(lambda x: print(x.shape),[train_high, train_low, test_high, test_low]))

(1615, 28)
(38730, 28)
(521, 27)
(12928, 27)


[None, None, None, None]

In [4]:
xTrainCols = train_high.col_names[1:-1]
xTrainCols

['carat',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_0',
 'cut_1',
 'cut_2',
 'cut_3',
 'cut_4',
 'color_1',
 'color_2',
 'color_3',
 'color_4',
 'color_5',
 'color_6',
 'color_7',
 'clarity_0',
 'clarity_1',
 'clarity_2',
 'clarity_3',
 'clarity_4',
 'clarity_5',
 'clarity_6',
 'clarity_7']

In [5]:
train_frame = train_high
model_name = 'diamonds_high'

aml_high = H2OAutoML(max_runtime_secs = 300, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_high.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_high = aml_high.leader.predict(test_high)
display(pred_high)
display(aml_high.leaderboard)

AutoML progress: |████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
16400.8
16533.3
15796.1
16969
15502
13281.1
16728.2
17059.1
14953.6
15944.6


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200301_130442,2.75587e+06,1660.08,2.75587e+06,1290.89,0.118841
XGBoost_1_AutoML_20200301_130442,2.80209e+06,1673.94,2.80209e+06,1312.13,0.120124
XGBoost_2_AutoML_20200301_130442,2.80446e+06,1674.65,2.80446e+06,1305.19,0.120139
GBM_4_AutoML_20200301_130442,2.83699e+06,1684.34,2.83699e+06,1309.41,0.120974
XGBoost_3_AutoML_20200301_130442,2.8408e+06,1685.47,2.8408e+06,1331.35,0.120487
GBM_1_AutoML_20200301_130442,2.84172e+06,1685.74,2.84172e+06,1318.94,0.121426
GBM_grid__1_AutoML_20200301_130442_model_6,2.85545e+06,1689.81,2.85545e+06,1319.18,0.122458
GBM_3_AutoML_20200301_130442,2.88136e+06,1697.46,2.88136e+06,1325.21,0.123306
GBM_2_AutoML_20200301_130442,2.88282e+06,1697.89,2.88282e+06,1340.24,0.121859
XGBoost_grid__1_AutoML_20200301_130442_model_3,2.91574e+06,1707.55,2.91574e+06,1328.71,0.12217


In [6]:
tableHighPreds = test_high['id'].cbind(pred_high)
tableHighPreds = tableHighPreds.rename(columns={'predict':'price'})

In [7]:
train_frame = train_low
model_name = 'diamonds_low'

aml_low = H2OAutoML(max_runtime_secs = 300, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_low.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_low = aml_low.leader.predict(test_low)
display(pred_low)
display(aml_low.leaderboard)

AutoML progress: |████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
4463.72
1028.93
7017.93
1468.14
2205.22
1972.98
1828.71
3479.06
15289.9
3693.56


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200301_130940,182514,427.217,182514,225.51,0.0898556
StackedEnsemble_BestOfFamily_AutoML_20200301_130940,183520,428.392,183520,227.239,0.091634
XGBoost_1_AutoML_20200301_130940,184459,429.487,184459,229.456,0.0929842
XGBoost_2_AutoML_20200301_130940,190613,436.592,190613,227.187,0.0884152
DRF_1_AutoML_20200301_130940,212211,460.663,212211,243.342,0.0981872
XGBoost_3_AutoML_20200301_130940,234841,484.604,234841,269.462,0.119303
GBM_1_AutoML_20200301_130940,348515,590.351,348515,338.824,0.168517
GBM_2_AutoML_20200301_130940,2.24624e+06,1498.75,2.24624e+06,1073.34,0.584989
GBM_3_AutoML_20200301_130940,9.12739e+06,3021.16,9.12739e+06,2320.8,0.986124
GLM_1_AutoML_20200301_130940,1.11047e+07,3332.36,1.11047e+07,2572.5,1.05989


In [8]:
tableLowPreds = test_low['id'].cbind(pred_low)
tableLowPreds = tableLowPreds.rename(columns={'predict':'price'})

In [9]:
combinedTable = tableLowPreds.concat(tableHighPreds, axis=0)
combinedTable.shape

(13449, 2)

In [10]:
h2o.export_file(frame=combinedTable,path='../predictions/12-Auto-ML-Split-data-Dummies-compl-data.csv')

Export File progress: |███████████████████████████████████████████████████| 100%
